In [5]:
import numpy as np
import pandas as pd

def calculate_individual_stock_alpha():
    # Stock tickers for reference
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    # Individual stock returns
    q1_returns = np.array([-1.5, -12.2, 7.0, 18.1, -34.9, -7.5, 6.5, -27.6, -11.8, 7.5,
                          -10.2, 2.6, -17.6, 6.7, -21.9, -25.9, 13.7, 16.5, -2.1, -13.1,
                          -37.9, -68.4, -21.8, -13.2, 30.1, 3.8, -22.4, -3.6, -10.1])

    q2_returns = np.array([22, -21, -28, 0, 17, 9, 8, 24, 21, 24, 9, 11, 28, -6, -1, 26,
                          -11, -1, 19, 19, -1, 17, 12, 16, 50, -67, 12, 14, 23])

    # Market returns (S&P 500)
    market_q1 = -4.6
    market_q2 = 9.0  # Estimated

    # Risk-free rate
    risk_free_rate = 4.42 / 4  # Quarterly

    # Calculate alpha for each stock
    alphas_q1 = []
    alphas_q2 = []
    avg_alphas = []

    for i, ticker in enumerate(tickers):
        # Alpha = (Stock Return - Risk-Free Rate) - (Market Return - Risk-Free Rate)
        stock_excess_q1 = q1_returns[i] - risk_free_rate
        market_excess_q1 = market_q1 - risk_free_rate
        alpha_q1 = stock_excess_q1 - market_excess_q1

        stock_excess_q2 = q2_returns[i] - risk_free_rate
        market_excess_q2 = market_q2 - risk_free_rate
        alpha_q2 = stock_excess_q2 - market_excess_q2

        avg_alpha = (alpha_q1 + alpha_q2) / 2

        alphas_q1.append(alpha_q1)
        alphas_q2.append(alpha_q2)
        avg_alphas.append(avg_alpha)

    # Create original order DataFrame
    df_original = pd.DataFrame({
        'Stock': tickers,
        'Q1_Alpha': alphas_q1,
        'Q2_Alpha': alphas_q2,
        'Avg_Alpha': avg_alphas
    })

    # Create sorted DataFrame
    df_sorted = df_original.sort_values('Avg_Alpha', ascending=False)

    # Save to CSV files
    df_original.to_csv('alpha_original_order.csv', index=False)
    df_sorted.to_csv('alpha_sorted_by_performance.csv', index=False)

    return df_sorted, df_original

def calculate_portfolio_alpha():
    # Portfolio weights
    weights = np.array([13.9407, 9.7876, 7.4736, 6.0693, 6.0428, 5.7082, 5.6708, 4.9494,
                       3.7019, 3.5918, 3.2948, 3.0735, 2.9403, 2.7125, 2.6726, 2.4349,
                       2.0997, 2.0104, 1.7872, 1.7506, 1.7250, 1.7083, 1.4630, 1.2205,
                       0.8973, 0.6003, 0.5892, 0.0836, 0.0000]) / 100

    # Portfolio returns
    q1_returns = np.array([-1.5, -12.2, 7.0, 18.1, -34.9, -7.5, 6.5, -27.6, -11.8, 7.5,
                          -10.2, 2.6, -17.6, 6.7, -21.9, -25.9, 13.7, 16.5, -2.1, -13.1,
                          -37.9, -68.4, -21.8, -13.2, 30.1, 3.8, -22.4, -3.6, -10.1])

    q2_returns = np.array([22, -21, -28, 0, 17, 9, 8, 24, 21, 24, 9, 11, 28, -6, -1, 26,
                          -11, -1, 19, 19, -1, 17, 12, 16, 50, -67, 12, 14, 23])

    # Calculate portfolio returns
    q1_portfolio_return = np.sum(weights * q1_returns)
    q2_portfolio_return = np.sum(weights * q2_returns)

    # Market returns (S&P 500)
    market_q1 = -4.6
    market_q2 = 9.0

    # Risk-free rate
    risk_free_rate = 4.42 / 4

    # Calculate portfolio alpha
    portfolio_excess_q1 = q1_portfolio_return - risk_free_rate
    market_excess_q1 = market_q1 - risk_free_rate
    portfolio_alpha_q1 = portfolio_excess_q1 - market_excess_q1

    portfolio_excess_q2 = q2_portfolio_return - risk_free_rate
    market_excess_q2 = market_q2 - risk_free_rate
    portfolio_alpha_q2 = portfolio_excess_q2 - market_excess_q2

    avg_portfolio_alpha = (portfolio_alpha_q1 + portfolio_alpha_q2) / 2

    print("Portfolio Alpha Analysis:")
    print(f"Q1 Portfolio Alpha: {portfolio_alpha_q1:.4f}%")
    print(f"Q2 Portfolio Alpha: {portfolio_alpha_q2:.4f}%")
    print(f"Average Portfolio Alpha: {avg_portfolio_alpha:.4f}%")

    return {
        'q1_alpha': portfolio_alpha_q1,
        'q2_alpha': portfolio_alpha_q2,
        'avg_alpha': avg_portfolio_alpha
    }

def calculate_market_bottom_alpha():
    # Stock tickers for reference
    tickers = ['META', 'REGN', 'FI', 'GILD', 'MITK', 'FIS', 'IDCC', 'COUR', 'AMAT', 'KLAC',
               'NICE', 'WFC', 'AMD', 'LLY', 'NVO', 'DELL', 'BP', 'NOK', 'RMBS', 'EL',
               'UIS', 'ILMN', 'GOOG', 'SNPS', 'GRAL', 'UNH', 'GNRC', 'TXN', 'ROK']

    # Portfolio weights
    weights = np.array([13.9407, 9.7876, 7.4736, 6.0693, 6.0428, 5.7082, 5.6708, 4.9494,
                       3.7019, 3.5918, 3.2948, 3.0735, 2.9403, 2.7125, 2.6726, 2.4349,
                       2.0997, 2.0104, 1.7872, 1.7506, 1.7250, 1.7083, 1.4630, 1.2205,
                       0.8973, 0.6003, 0.5892, 0.0836, 0.0000]) / 100

    # Returns since market bottom (April 8 - July 8, 2025)
    market_bottom_returns = np.array([41, -1, -10, 9, 46, 21, 18, 38, 48, 55, 19, 35, 76, 7, 12, 74,
                                     20, 12, 53, 79, 22, 43, 23, 44, 116, -44, 49, 49, 59])

    # Market return since bottom (estimated S&P 500)
    market_return_since_bottom = 22.0  # Estimated based on recovery

    # Risk-free rate for 3-month period
    risk_free_rate = 4.42 * (3/12)  # 3-month period

    # Calculate alpha for each stock
    alphas_market_bottom = []

    for i, ticker in enumerate(tickers):
        # Alpha = (Stock Return - Risk-Free Rate) - (Market Return - Risk-Free Rate)
        stock_excess = market_bottom_returns[i] - risk_free_rate
        market_excess = market_return_since_bottom - risk_free_rate
        alpha = stock_excess - market_excess

        alphas_market_bottom.append(alpha)

    # Create DataFrame
    df_market_bottom = pd.DataFrame({
        'Stock': tickers,
        'Market_Bottom_Alpha': alphas_market_bottom
    })

    # Sort by alpha
    df_market_bottom_sorted = df_market_bottom.sort_values('Market_Bottom_Alpha', ascending=False)

    # Calculate portfolio alpha since market bottom
    portfolio_return = np.sum(weights * market_bottom_returns)
    portfolio_excess = portfolio_return - risk_free_rate
    market_excess = market_return_since_bottom - risk_free_rate
    portfolio_alpha = portfolio_excess - market_excess

    print("Alpha Since Market Bottom (April 8 - July 8, 2025):")
    print(f"Portfolio Alpha: {portfolio_alpha:.4f}%")
    print(f"Market Return: {market_return_since_bottom:.2f}%")
    print(f"Portfolio Return: {portfolio_return:.4f}%")

    # Save to CSV
    df_market_bottom.to_csv('alpha_market_bottom_original.csv', index=False)
    df_market_bottom_sorted.to_csv('alpha_market_bottom_sorted.csv', index=False)

    return df_market_bottom_sorted, df_market_bottom, portfolio_alpha

if __name__ == "__main__":
    print("Individual Stock Alpha Analysis")
    print("=" * 50)

    alpha_df_sorted, alpha_df_original = calculate_individual_stock_alpha()

    print("\nIndividual Stock Alpha Analysis (Sorted by Performance):")
    print(alpha_df_sorted.to_string(index=False, float_format='%.2f'))

    print("\nIndividual Stock Alpha Analysis (Original Order):")
    print(alpha_df_original.to_string(index=False, float_format='%.2f'))

    print("\n" + "=" * 50)

    portfolio_alpha = calculate_portfolio_alpha()

    print("\n" + "=" * 50)

    market_bottom_sorted, market_bottom_original, portfolio_alpha_mb = calculate_market_bottom_alpha()

    print("\nAlpha Since Market Bottom (Sorted by Performance):")
    print(market_bottom_sorted.to_string(index=False, float_format='%.2f'))



Individual Stock Alpha Analysis

Individual Stock Alpha Analysis (Sorted by Performance):
Stock  Q1_Alpha  Q2_Alpha  Avg_Alpha
 GRAL     34.70     41.00      37.85
 KLAC     12.10     15.00      13.55
 META      3.10     13.00       8.05
 GILD     22.70     -9.00       6.85
 RMBS      2.50     10.00       6.25
  NOK     21.10    -10.00       5.55
 IDCC     11.10     -1.00       5.05
  WFC      7.20      2.00       4.60
  ROK     -5.50     14.00       4.25
  TXN      1.00      5.00       3.00
  AMD    -13.00     19.00       3.00
 AMAT     -7.20     12.00       2.40
   EL     -8.50     10.00       0.75
 SNPS     -8.60      7.00      -0.80
   BP     18.30    -20.00      -0.85
  FIS     -2.90      0.00      -1.45
  LLY     11.30    -15.00      -1.85
 DELL    -21.30     17.00      -2.15
 NICE     -5.60      0.00      -2.80
 COUR    -23.00     15.00      -4.00
 GOOG    -17.20      3.00      -7.10
 GNRC    -17.80      3.00      -7.40
 MITK    -30.30      8.00     -11.15
   FI     11.60    -37